# 🚀 Pinokio Cloud GPU Deployment

This notebook deploys the Pinokio AI tool environment on cloud GPU platforms with automatic platform detection, tunneling, and AI tool management.

**Supported Platforms:**
- Google Colab
- Kaggle
- Paperspace Gradient
- Lightning.ai
- RunPod
- Vast.ai
- DigitalOcean
- AWS SageMaker
- GCP Vertex AI

## 📦 Step 1: Clone Repository and Install Dependencies

In [ ]:
import os
import sys
import subprocess

# Clone the Pinokio Cloud GPU repository
repo_url = "https://github.com/remphanostar/SD-LongNose.git"
repo_dir = "SD-LongNose"

if not os.path.exists(repo_dir):
    print("📥 Cloning repository...")
    subprocess.run(["git", "clone", repo_url], check=True)
    print("✅ Repository cloned")
else:
    print("✅ Repository already exists")

# Add to Python path
sys.path.insert(0, repo_dir)

## 📦 Step 2: Install Required Packages

In [ ]:
# Install required packages
packages = [
    "requests",
    "pyngrok",
    "psutil",
    "cloud-detect"
]

for package in packages:
    print(f"Installing {package}...")
    subprocess.run([sys.executable, "-m", "pip", "install", "-q", package], check=False)

print("✅ Dependencies installed")

## 🔍 Step 3: Platform Detection and Setup

In [ ]:
# Import modules
from modules.platform_detector import PlatformDetector
from modules.pinokio_installer import PinokioInstaller
from modules.tunnel_manager import TunnelManager
from modules.pinokio_controller import PinokioController

# Detect platform
detector = PlatformDetector()
platform = detector.detect_platform()
paths = detector.setup_paths()
gpu_info = detector.detect_gpu()

# Print system information
detector.print_summary()

## 💾 Step 4: Mount Persistent Storage (Colab Only)

In [ ]:
# Mount Google Drive for persistent storage (Colab only)
if platform == "colab":
    detector.mount_google_drive()

## 🎯 Step 5: Install Pinokio

In [ ]:
# Create directories
detector.create_directories()

# Install Pinokio
installer = PinokioInstaller(paths['pinokio'])
installer.install(headless=True)

## 🚀 Step 6: Start Pinokio Server

In [ ]:
# Start Pinokio controller
controller = PinokioController(paths['pinokio'])
controller.start_server(headless=True, port=42000)

## 🌐 Step 7: Setup Tunneling

**Note:** For ngrok, you'll need an auth token from https://ngrok.com/

In [ ]:
# Setup tunnel manager
tunnel = TunnelManager()

# Option 1: Use ngrok (requires auth token)
# ngrok_token = "YOUR_NGROK_AUTH_TOKEN"  # Replace with your token
# tunnel_url = tunnel.start_ngrok_tunnel(42000, ngrok_token)

# Option 2: Use Cloudflare Tunnel (no auth required)
tunnel_url = tunnel.start_cloudflare_tunnel(42000)

# Option 3: Auto-select best available
# tunnel_url = tunnel.start_tunnel(42000, service="auto")

if tunnel_url:
    print(f"\n🎉 Pinokio is accessible at: {tunnel_url}")
    print(f"📱 Open this URL in your browser to access the Pinokio interface")
else:
    print("⚠️ Tunneling failed - Pinokio is only accessible locally")

## 🤖 Step 8: Install AI Tools (Optional)

You can install popular AI tools directly through the API:

In [ ]:
# Example: Install and launch Stable Diffusion WebUI
tool_url = controller.quick_install_and_launch("stable-diffusion")
if tool_url:
    print(f"🎨 Stable Diffusion WebUI is running at: {tool_url}")

# List available apps
# apps = controller.list_apps()

# Install specific tool
# controller.install_app("https://github.com/AUTOMATIC1111/stable-diffusion-webui", "sd-webui")

# Launch installed tool
# result = controller.launch_app("sd-webui")

## 🎯 Quick Start Function

Use this cell for a one-click setup:

In [ ]:
def quick_start(ngrok_token=None, install_tools=None):
    """
    Quick start Pinokio with auto-configuration
    
    Args:
        ngrok_token: Optional ngrok auth token for tunneling
        install_tools: List of tools to install (e.g., ["stable-diffusion", "comfyui"])
    """
    from pinokio_cloud_main import PinokioCloudGPU
    
    # Initialize and setup
    pinokio = PinokioCloudGPU()
    
    # Setup environment
    pinokio.setup()
    
    # Install Pinokio
    pinokio.install_pinokio()
    
    # Start server
    pinokio.start_pinokio()
    
    # Setup tunnel
    tunnel_url = pinokio.setup_tunnel(ngrok_token=ngrok_token)
    
    # Install tools if specified
    if install_tools:
        for tool in install_tools:
            pinokio.install_ai_tool(tool)
    
    return pinokio, tunnel_url

# Run quick start
# pinokio, url = quick_start()
# print(f"\n🎉 Access Pinokio at: {url}")

## 🛑 Step 9: Cleanup (Run when finished)

In [ ]:
# Stop all services
def cleanup():
    """Stop all services and clean up"""
    try:
        # Stop AI apps
        # controller.stop_app("sd-webui")
        
        # Stop Pinokio server
        controller.stop_server()
        
        # Stop tunnels
        tunnel.stop_all_tunnels()
        
        print("✅ All services stopped")
    except Exception as e:
        print(f"⚠️ Cleanup error: {e}")

# Uncomment to run cleanup
# cleanup()

## 📚 Additional Resources

### Available AI Tools:
- **stable-diffusion** - Stable Diffusion WebUI by AUTOMATIC1111
- **comfyui** - ComfyUI for node-based workflows
- **text-generation** - Text Generation WebUI by oobabooga
- **kohya-ss** - Kohya SS for model training
- **invokeai** - InvokeAI for advanced image generation

### Useful Commands:
```python
# Check system info
detector.get_system_info()

# List installed apps
controller.list_apps()

# Check app status
controller.get_app_status("app-name")

# View active tunnels
tunnel.get_active_tunnels()
```

### Troubleshooting:
1. **GPU not detected**: Check platform GPU allocation
2. **Tunnel fails**: Try different service (ngrok, cloudflare, localtunnel)
3. **Pinokio won't start**: Check logs in `paths['pinokio']/logs/`
4. **Out of memory**: Reduce model size or use CPU mode

### Support:
- GitHub Issues: [Report bugs and request features](https://github.com/remphanostar/SD-LongNose)
- Documentation: [Full documentation](https://github.com/remphanostar/SD-LongNose/wiki)